<a href="https://colab.research.google.com/github/dataforgoodfr/batch8_mednum/blob/master/notebooks/acces_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Liste des médiathèques


In [14]:
# get mediatheques map
import pandas as pd
import numpy as np
!pip install pgeocode

In [7]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [22]:
import pgeocode
dist = pgeocode.GeoDistance('fr')

!rm -rf batch8_mednum/
!git clone https://github.com/dataforgoodfr/batch8_mednum.git
!mv batch8_mednum/data/table_iris_insee_epci.csv .
!ls

#commune = pd.read_csv('table_iris_insee_epci.csv')
#del commune['Unnamed: 0']
commune = pd.read_csv('/content/drive/My Drive/Colab Notebooks/laposte_hexasmal.csv', delimiter=';')
commune.rename(columns={'Code_postal': 'code_postal'}, inplace=True)
code_postal = []
for value in commune["code_postal"]:
    value = str(value)
    code_postal.append(value.zfill(5))
len(code_postal)
commune["code_postal"] = code_postal

Cloning into 'batch8_mednum'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 120 (delta 60), reused 30 (delta 8), pack-reused 0
Receiving objects: 100% (120/120), 5.30 MiB | 6.79 MiB/s, done.
Resolving deltas: 100% (60/60), done.
batch8_mednum  France-services		sample_data
drive	       france_services.geojson	table_iris_insee_epci.csv


In [23]:
commune

,Code_commune_INSEE,Nom_commune,code_postal,Ligne_5,Libellé_d_acheminement,coordonnees_gps
0,01004,AMBERIEU EN BUGEY,01500,NaN,AMBERIEU EN BUGEY,"45.9608475114,5.3729257777"
1,01008,AMBUTRIX,01500,NaN,AMBUTRIX,"45.9367134524,5.3328092349"
2,01015,ARBOYS EN BUGEY,01300,ST BOIS,ARBOYS EN BUGEY,"45.7237621545,5.65263086429"
3,01017,ARGIS,01230,NaN,ARGIS,"45.9337182132,5.48251100314"
4,01019,ARMIX,01510,NaN,ARMIX,"45.8541838459,5.58357838695"
...,...,...,...,...,...,...
39187,98824,POUEBO,98824,NaN,POUEBO,NaN
39188,98825,POUEMBOUT,98825,NaN,POUEMBOUT,NaN
39189,98828,SARRAMEA,98882,NaN,SARRAMEA,NaN
39190,98830,TOUHO,98831,NaN,TOUHO,NaN


In [41]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/carte-des-bibliotheques.csv', header=0, sep=';')
df.rename(columns={'Code postal': 'code_postal'}, inplace=True)
print(df.shape)
df.head()

(844, 21)


,Nom structure,Adresse,code_postal,Département,Ville,Année(s) subvention Région,Montant subvention 2000,Montant subvention 2001,Montant subvention 2002,Montant subvention 2003,Montant subvention 2004,Montant subvention 2005,Montant subvention 2006,Montant subvention 2007,Montant subvention 2008,Montant subvention 2009,Montant subvention 2010,Montant subvention 2011,Montant subvention 2012,Montant subvention 2013,wgs84
0,MÉDIATHÈQUE SAINT-JOHN PERSE,2 rue Édouard Poisson,93300,93,AUBERVILLIERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.9104747,2.385008"
1,MÉDIATHÈQUE GEORGES PÉREC,8-10 rue du Marqis de Raies,91080,91,COURCOURONNES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.6293899,2.4135832"
2,MÉDIATHÈQUE LES MUSES,101 rue de la Division Leclerc,91310,91,LINAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.6305113,2.2650636"
3,MÉDIATHÈQUE MUNICIPALE,2 route de la Creusée,91530,91,LE-VAL-SAINT-GERMAIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.5657217,2.0601725"
4,BIBLIOTHÈQUE LE GRENIER,5 place des Droits de l'Homme,91800,91,BOUSSY-SAINT-ANTOINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.6891076,2.5297452"


In [42]:
df = df[['code_postal', 'Ville', 'Département', 'Nom structure']]
df.rename(columns={'Ville': 'Ville_bibliotheques'}, inplace=True)
df['code_postal'] = df['code_postal'].astype(str)
df.head()

,code_postal,Ville_bibliotheques,Département,Nom structure
0,93300,AUBERVILLIERS,93,MÉDIATHÈQUE SAINT-JOHN PERSE
1,91080,COURCOURONNES,91,MÉDIATHÈQUE GEORGES PÉREC
2,91310,LINAS,91,MÉDIATHÈQUE LES MUSES
3,91530,LE-VAL-SAINT-GERMAIN,91,MÉDIATHÈQUE MUNICIPALE
4,91800,BOUSSY-SAINT-ANTOINE,91,BIBLIOTHÈQUE LE GRENIER


In [19]:
# mapping code postal - code INSEE
#mapping = pd.read_csv('/content/drive/My Drive/Colab Notebooks/correspondances-code-insee-code-postal.csv', header=0, sep=';')
#print(mapping.shape)
#mapping.rename(columns={'Code Postal': 'Code postal'}, inplace=True)
#mapping.drop_duplicates(inplace=True)
#mapping.head()

In [53]:
biblio = commune[['Code_commune_INSEE', 'Nom_commune', 'code_postal']].merge(df, on='code_postal', how='outer',).dropna(subset=['Code_commune_INSEE']).reset_index(drop=True)
biblio

,Code_commune_INSEE,Nom_commune,code_postal,Ville_bibliotheques,Département,Nom structure
0,01004,AMBERIEU EN BUGEY,01500,NaN,NaN,NaN
1,01008,AMBUTRIX,01500,NaN,NaN,NaN
2,01007,AMBRONAY,01500,NaN,NaN,NaN
3,01089,CHATEAU GAILLARD,01500,NaN,NaN,NaN
4,01379,ST MAURICE DE REMENS,01500,NaN,NaN,NaN
...,...,...,...,...,...,...
40892,98824,POUEBO,98824,NaN,NaN,NaN
40893,98825,POUEMBOUT,98825,NaN,NaN,NaN
40894,98828,SARRAMEA,98882,NaN,NaN,NaN
40895,98830,TOUHO,98831,NaN,NaN,NaN


In [56]:
dir = '/content/drive/My Drive/Colab Notebooks/'
biblio.to_csv(f'{dir}data_mediatheques.csv', index=False)

## 2. Liste des centres sociaux

In [1]:
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib import request

import time


def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()        
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result    
    return timed

In [2]:
@timeit
def scrap_table_page(url='https://www.senacs.fr/structure/csx?page=', page_number=1):
    """
    Generate a request on senacs page for a given 
    page number to get the name and location of 
    each social center.
    """

    url_page = url + str(page_number)
    request_text = request.urlopen(url_page).read()
    page = BeautifulSoup(request_text, 'html.parser')

    table = page.find('table', {'class': 'table table-striped table-hover'}).findAll('td')

    # fill the values of id, name and city
    ids = [id.getText() for id in table[::4]]
    names = [name.getText() for name in table[1::4]]
    cities = [city.getText() for city in table[2::4]]

    df = pd.DataFrame({'id': ids, 
                      'nom': names,
                      'ville': cities}
                      )
    df['page'] = page_number

    return df


def stack_pages(pages=range(91)):
    """
    Concatenate all empruntis scrapped data per region.
    """

    data = pd.DataFrame()    # empty Dataframe to fill
    for page_number in pages:
        temp = scrap_table_page(page_number=page_number)
        data = pd.concat([data, temp], axis=0)

    return data

In [ ]:
data = stack_pages()

In [4]:
data['code_postal'] = data['ville'].apply(lambda x: x.split('-')[0])
data['ville'] = data['ville'].apply(lambda x: x.split('-')[1])

In [5]:
data

,id,nom,ville,page,code_postal
0,91,CENTRE SOCIAL DES GRANDES BORNES,Goussainville,0,95190
1,92,CENTRE SOCIAL ARCHIPELIA,PARIS 20EME ARRONDISSEMENT,0,75020
2,93,Espace Socioculturel Val de Charente,Ruffec,0,16700
3,94,CENTRE SOCIAL Maison Des Habitants Champaret,Bourgoin,0,38300
4,95,CENTRE SOCIAL MAISON DE LA CHALLE,Éragny,0,95610
...,...,...,...,...,...
6,16264,CENTRE SOCIOCULTUREL MUNICIPAL DE SAINT-DIZIER,Saint,90,52100
7,16267,ESPACE DE VIE SOCIALE L.E.P.H.A.R.E,Aniche,90,59580
8,16351,ASSOCIATION FERME BEAUREPAIRE,Boulogne,90,62200
9,16355,OFFICE DE LA JEUNESSE,Bruay,90,62700


In [8]:
dir = '/content/drive/My Drive/Colab Notebooks/'
data.to_csv(f'{dir}data_centre_sociaux.csv', index=False)

## 3. Distance des communes à l'accès le plus proche


In [9]:
# copy geoson file with every 'Franceservices' locations

import os

if 'COLAB_GPU' in os.environ:  # this is always set on Colab, the value is 0 or 1 depending on whether a GPU is attached
    from google.colab import auth
    #auth.authenticate_user()

    !rm -rf France-services/ 
    !git clone https://github.com/cget-carto/France-services.git
    !mv France-services/data/france_services.geojson .

Cloning into 'France-services'...
remote: Enumerating objects: 260, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 260 (delta 108), reused 221 (delta 78), pack-reused 0
Receiving objects: 100% (260/260), 2.12 MiB | 11.94 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [10]:
import pandas as pd
import json

with open("france_services.geojson", "r") as read_file: 
    fs = json.load(read_file) 

In [11]:
data_fs = pd.DataFrame()
for n, temp in enumerate(fs['features']): 
 
    df_temp = pd.DataFrame.from_records([{'latitude': temp['properties']['LATITUDE'], 
                            'longitude': temp['properties']['LATITUDE'], 
                            'departement': temp['properties']['DEPARTEMENT'],
                            'insee_com': temp['properties']['insee_com'],
                            'code_postal': temp['properties']['code_postal'], 
                            'lib_france_services': temp['properties']['lib_france_services']
                            }])
    
    data_fs = pd.concat([data_fs, df_temp], axis=0)

In [12]:
data_fs

,latitude,longitude,departement,insee_com,code_postal,lib_france_services
0,46.105747,46.105747,1,01033,01200,Valserhône
0,44.031548,44.031548,12,12197,12170,Réquista « Le Bercail »
0,44.475185,44.475185,12,12138,12330,Conques-Marcillac
0,49.291400,49.291400,14,14514,14130,Terre d’Auge
0,49.126169,49.126169,14,14057,14370,Val ès Dunes
...,...,...,...,...,...,...
0,47.617932,47.617932,89,89368,89520,Saint-Sauveur-en-Puisaye
0,48.980007,48.980007,95,95219,95120,Ermont
0,47.181000,47.181000,25,25527,25410,Saint Vit
0,43.208769,43.208769,31,31375,31310,Volvestre – Antenne Montesquieu


### Calculer la distance entre le centre des communes et la prefecture, sous-prefecture ou france-services la plus proche en km. 

- scrapper prefecture / sous-prefecture https://fr.wikipedia.org/wiki/Liste_des_pr%C3%A9fectures_de_France
- obtenir coordonnées des communes: OK via codes postaux + librairie `pgecode`

In [18]:
#data['distance'] = dist.query_postal_code(data[''].values, data[''].values)

# calculer le france-service (ou prefecture, sous-prefecture etc.) le plus proche de chaque commune et bueno 

In [24]:
def df_crossjoin(df1, df2, **kwargs):
    """
    Make a cross join (cartesian product) between two dataframes by using a constant temporary key.
    Also sets a MultiIndex which is the cartesian product of the indices of the input dataframes.
    See: https://github.com/pydata/pandas/issues/5401
    :param df1 dataframe 1
    :param df1 dataframe 2
    :param kwargs keyword arguments that will be passed to pd.merge()
    :return cross join of df1 and df2
    """
    df1['_tmpkey'] = 1
    df2['_tmpkey'] = 1

    res = pd.merge(df1, df2, on='_tmpkey', **kwargs).drop('_tmpkey', axis=1)
    res.index = pd.MultiIndex.from_product((df1.index, df2.index))

    df1.drop('_tmpkey', axis=1, inplace=True)
    df2.drop('_tmpkey', axis=1, inplace=True)

    return res

df = df_crossjoin(commune[['code_postal']], data_fs[['code_postal']], suffixes=('_communes', '_fs')).reset_index()
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

,level_0,level_1,code_postal_communes,code_postal_fs
0,0,0,01500,01200
1,0,0,01500,12170
2,0,0,01500,12330
3,0,0,01500,14130
4,0,0,01500,14370
...,...,...,...,...
35586331,39191,0,98799,89520
35586332,39191,0,98799,95120
35586333,39191,0,98799,25410
35586334,39191,0,98799,31310


In [27]:
df.drop_duplicates()

,level_0,level_1,code_postal_communes,code_postal_fs
0,0,0,01500,01200
1,0,0,01500,12170
2,0,0,01500,12330
3,0,0,01500,14130
4,0,0,01500,14370
...,...,...,...,...
35586331,39191,0,98799,89520
35586332,39191,0,98799,95120
35586333,39191,0,98799,25410
35586334,39191,0,98799,31310


In [39]:
df[:len(data_fs)].groupby('code_postal_communes').apply(lambda row: dist.query_postal_code(row['code_postal_communes'], row['code_postal_fs']))

code_postal_communes
01500    []
dtype: object

In [40]:
df[:len(data_fs)].apply(lambda row: dist.query_postal_code(row['code_postal_communes'], row['code_postal_fs']), axis=1)

0       42.623548
1      301.434162
2      275.230403
3      527.270586
4      544.313797
          ...    
903    238.162672
904    409.646169
905    143.125315
906    446.226215
907    431.109707
Length: 908, dtype: float64

In [41]:
df['distance_commune_france_service'] = df.apply(lambda row: dist.query_postal_code(row['code_postal_communes'], row['code_postal_fs']), axis=1)

KeyboardInterrupt: ignored

In [ ]:
# take the minimum per commune and OK
# df.groupby('code_postal_communes').agg({'distance_commune_france_service': ['mean', 'min']})

In [42]:
df

,level_0,level_1,code_postal_communes,code_postal_fs
0,0,0,01500,01200
1,0,0,01500,12170
2,0,0,01500,12330
3,0,0,01500,14130
4,0,0,01500,14370
...,...,...,...,...
35586331,39191,0,98799,89520
35586332,39191,0,98799,95120
35586333,39191,0,98799,25410
35586334,39191,0,98799,31310


## 4. Part des moins de 25 ans

In [80]:
demo1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_DEMO_2020_V1_COM.csv', sep=';')
demo2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_DEMO_2020_V1_ENSQP.csv', sep=';')
demo3 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_DEMO_2020_V1_EPCI15.csv', sep=';')
demo4 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_DEMO_2020_V1_EPCI19.csv', sep=';')
demo5 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_DEMO_2020_V1_QP.csv', sep=';')

demo = pd.concat([demo1,demo2,demo3,demo4,demo5], axis=0).reset_index()
demo = demo[['CODGEO', 'POP_MUN', 'TX_TOT_0A24']]
#demo.rename(columns={'CODGEO': 'code_postal'}, inplace=True)
demo.dropna(subset=['TX_TOT_0A24'], inplace=True)
demo

,CODGEO,POP_MUN,TX_TOT_0A24
0,01004,14359,"32,5"
1,01031,3646,"32,1"
2,01034,8909,"29,5"
3,01053,40490,"31,3"
4,01160,9236,"29,1"
...,...,...,...
2899,QP973028,3111,"55,1"
2900,QP973029,4711,"60,4"
2901,QP973030,9155,"69,7"
2902,QP973031,3740,"55,6"


In [81]:
dir = '/content/drive/My Drive/Colab Notebooks/'
demo.to_csv(f'{dir}data_moins_25ans.csv', index=False)

In [ ]:
#pd.merge(commune, demo, how='outer', on='')